# Gaining intuition for our DAQ: 

In [1]:
import os
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Plt settings. 
plt.rcParams["figure.dpi"] = 200
plt.rcParams.update({"font.size": 12})
plt.rcParams['figure.figsize'] = (8,5)

In [2]:
# Constants. 
PI = np.pi

## 1. FFT of noisy sin wave: 

$ y(t) = A \, sin(2 \pi f t) + RandGauss(\mu = 0, \sigma = N_{std})$

In [12]:
@interact
def simple_fft(f = widgets.FloatSlider(value=150,min=0,max=500,step=10.0), A = (0,10,1), N_std = (0,10,1)): 
    
    # Alter names: 
    signal_amp = A
    noise_std = N_std
    # Discrete time axis. 
    signal_duration = 1
    fft_pts = 1000
    t = np.linspace(0, 1, fft_pts)

    # Create gaussian distributed noise with std noise_std. 
    noise = np.random.normal(0,noise_std,fft_pts)

    # Create signal.
    signal = signal_amp*np.sin(f * 2 * PI * t)

    # Add the noise. 
    signal = signal + noise
    
    fft = np.fft.fft(signal)
    T = t[1] - t[0]  # sampling interval 
    N = signal.size

    freq_axis = np.linspace(0, 1 / T, N)

    # Plot the signal and FFT
    fig, (ax1, ax2) = plt.subplots(2)
    
    ax1.set_title("Signal")
    ax1.set_ylabel("Amplitude")
    ax1.set_xlabel("Time [s]")
    ax1.plot(t, signal)
    
    ax2.set_title("FFT")
    ax2.set_ylabel("Amplitude")
    ax2.set_xlabel("Frequency [Hz]")
    ax2.bar(freq_axis[:N // 2], np.abs(fft)[:N // 2] * 1 / N, width=1)  # 1 / N is a normalization factor
    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=150.0, description='f', max=500.0, step=10.0), IntSlider(value=5, desc…

## 2. FFT of noisy modulated sin wave: 

$ y(t) = A \, sin(2 \pi f t + h \, sin(2 \pi f_m t ) )+ RandGauss(\mu = 0, \sigma = N_{std})$

In [13]:
@interact
def simple_fft(f = widgets.FloatSlider(value=150,min=0,max=500,step=10.0), A = (0,10,1),f_m = (0,60,5), h = (0,5,.1), N_std = (0,10,1)): 
    
    # Alter names: 
    signal_amp = A
    noise_std = N_std
    
    # Discrete time axis. 
    signal_duration = 1
    fft_pts = 1000
    t = np.linspace(0, 1, fft_pts)

    # Create gaussian distributed noise with std noise_std. 
    noise = np.random.normal(0,noise_std,fft_pts)

    # Create signal.
    
    signal = signal_amp*np.sin(f * 2 * PI * t+ h*np.sin(f_m * 2 * PI * t))

    # Add the noise. 
    signal = signal + noise
    
    fft = np.fft.fft(signal)
    T = t[1] - t[0]  # sampling interval 
    N = signal.size

    freq_axis = np.linspace(0, 1 / T, N)

    # Plot the signal and FFT
    fig, (ax1, ax2) = plt.subplots(2)
    
    ax1.set_title("Signal")
    ax1.set_ylabel("Amplitude")
    ax1.set_xlabel("Time [s]")
    ax1.plot(t, signal)
    
    ax2.set_title("FFT")
    ax2.set_ylabel("Amplitude")
    ax2.set_xlabel("Frequency [Hz]")
    ax2.bar(freq_axis[:N // 2], np.abs(fft)[:N // 2] * 1 / N, width=1)  # 1 / N is a normalization factor
    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=150.0, description='f', max=500.0, step=10.0), IntSlider(value=5, desc…

## 3. FFT of product of red and blue shifted waves:

$ y_1(t) = A_1 \, sin(2 \pi f t + h \, sin(2 \pi f_m t ) )+ RandGauss(\mu = 0, \sigma = N_{std1})$

$ y_2(t) = A_2 \, sin(2 \pi f t - h \, sin(2 \pi f_m t ) )+ RandGauss(\mu = 0, \sigma = N_{std2})$

In [16]:
@interact
def simple_fft(f = widgets.FloatSlider(value=150,min=0,max=500,step=10.0), A_1 = (0,10,1), A_2 = (0,10,1),
               f_m = (0,60,5), h = (0,5,.1), N_std1 = (0,10,1), N_std2 = (0,10,1), phi = (0,2*PI, .05)): 
    

    # Discrete time axis. 
    signal_duration = 1
    fft_pts = 1000
    t = np.linspace(0, 1, fft_pts)

    # Create gaussian distributed noise. 
    noise_1 = np.random.normal(0,N_std1,fft_pts)
    noise_2 = np.random.normal(0,N_std2,fft_pts)

    # Create two signals.

    signal_1 = A_1*np.sin(f * 2 * PI * t + h*np.sin(f_m * 2 * PI * t))
    signal_2 = A_2*np.sin(f * 2 * PI * t - h*np.sin(f_m * 2 * PI * t) + phi)

    # Add the noise. 
    signal_1 = signal_1 + noise_1
    signal_2 = signal_2 + noise_2

    # Create the product signal:
    signal = signal_1*signal_2
#     # Create signal.
    
#     signal = signal_amp*np.sin(f * 2 * PI * t+ h*np.sin(f_m * 2 * PI * t))

#     # Add the noise. 
#     signal = signal + noise
    fft_signal_1 = np.fft.fft(signal_1)
    fft_product = np.fft.fft(signal)
    
    T = t[1] - t[0]  # sampling interval 
    N = signal.size

    freq_axis = np.linspace(0, 1 / T, N)

    # Plot the signal and FFT
    fig, ( ax1, ax2) = plt.subplots(2)
     
    ax1.set_title("FFT of y_1(t) ")
    ax1.set_ylabel("Amplitude")
    ax1.set_xlabel("Frequency [Hz]")
    ax1.bar(freq_axis[:N // 2], np.abs(fft_signal_1)[:N // 2] * 1 / N, width=1) 
    
    ax2.set_title("FFT of Product Signal")
    ax2.set_ylabel("Amplitude")
    ax2.set_xlabel("Frequency [Hz]")
    ax2.bar(freq_axis[:N // 2], np.abs(fft_product)[:N // 2] * 1 / N, width=1) 
    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=150.0, description='f', max=500.0, step=10.0), IntSlider(value=5, desc…